<a href="https://colab.research.google.com/github/xinyangwo/recommender_system_education/blob/main/recommender_no_rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from google.colab import files
uploaded = files.upload()
df = pd.read_csv('Data2.csv')
df.head()
from ast import literal_eval
df['Choices'] = df['Choices'].apply(literal_eval)

# Combine choices into a single text column
df['CombinedChoices'] = df['Choices'].apply(lambda x: ' '.join(x))
x = df['CombinedChoices']

vectorizer = TfidfVectorizer(stop_words='english')  # Adjust as needed
X = vectorizer.fit_transform(df['CombinedChoices'])
# Train the model to predict cosine similarity as a binary classification problem
similarity_labels = cosine_similarity(X)

# Convert sparse matrix X to a TensorFlow sparse tensor
X_coo = X.tocoo()
X_reordered = tf.sparse.reorder(
    tf.SparseTensor(
        indices=np.vstack((X_coo.row, X_coo.col)).T,
        values=X_coo.data,
        dense_shape=X_coo.shape
    )
)

# Flatten upper triangular part and convert to a 1D array
y_train = similarity_labels[np.triu_indices(similarity_labels.shape[0], k=1)].reshape(-1, 1)

# Ensure that y_train has the same number of rows as X_reordered
y_train = y_train[:X_reordered.shape[0]]

# Convert y_train to a dense tensor
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Check the dimensions
print("X_reordered shape:", X_reordered.shape)
print("y_train_tensor shape:", y_train_tensor.shape)
# Update the model architecture to match the dimensions
model = Sequential()
model.add(Dense(8, input_shape=(X_reordered.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to see the architecture and parameter count
model.summary()

# Train the model
model.fit(X_reordered, y_train_tensor, epochs=50, batch_size=8)
import random

while True:
    # Randomly choose a question
    random_question_index = random.choice(df.index)
    random_question = df.loc[random_question_index, 'Question']
    choices = df.loc[random_question_index, 'Choices']
    correct_answer = df.loc[random_question_index, 'Correct Answer']

    # Print the random question and choices
    print("Question:", random_question)
    print("Choices:", choices)

    # Assume user provides an answer
    user_answer = input("Enter your answer: ")

    # Check if the user's answer is correct
    if user_answer == correct_answer:
        print("Your answer is correct!")
    else:
        print("Your answer is incorrect.")

        # Use the model to recommend a video link based on the user's incorrect answer
        user_input_vectorized = vectorizer.transform([random_question])
        user_input_reordered = tf.sparse.reorder(tf.SparseTensor(
            indices=np.vstack((user_input_vectorized.tocoo().row, user_input_vectorized.tocoo().col)).T,
            values=user_input_vectorized.tocoo().data,
            dense_shape=user_input_vectorized.tocoo().shape
        ))
        predicted_similarity = model.predict(user_input_reordered)

        # Find the most similar questions
        similar_questions_indices = np.argsort(predicted_similarity[:, 0])[::-1]  # Sort in descending order
        top_k = 1  # Recommend the top 1 question
        top_question_index = similar_questions_indices[0]

        # Get the recommended video link based on the similar question
        recommended_video_link = df.loc[top_question_index, 'VidioLink']  # Note: correct the typo in 'VidioLink'
        print(f"Recommended Video Link for Incorrect Answer: {recommended_video_link}")

    # Ask if the user wants to continue
    continue_playing = input("Do you want to continue? (yes/no): ").lower()
    if continue_playing != 'yes':
        print("Exiting...")
        break

# The code below is to save your model as a .h5 file.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model.save("recommender_no_rev.h5")

Saving Data2.csv to Data2 (1).csv
X_reordered shape: (30, 279)
y_train_tensor shape: (30, 1)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 2240      
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 2249 (8.79 KB)
Trainable params: 2249 (8.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
4/4 [==============================] - 1s 4ms/step - loss: 0.6857 - accuracy: 0.5333
Epoch 2/50
4/4 [==============================] - 0s 4ms/step - loss: 0.6735 - accuracy: 0.7000
Epoch 3/50
4/4 [==============================] - 0s 4ms/step - loss: 0.6626 - accuracy: 0.7667
Epoch 4/50
4/4 [=====================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('recommender_no_rev.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Save your model
import joblib
joblib.dump(model, 'recommender_no_rev.pkl')
print("Model dumped!")

Model dumped!


In [ ]:
model_columns = list(df.columns)
joblib.dump(model_columns, 'recommender_columns.pkl')
print("Model columns dumped!")

Model columns dumped!


In [ ]:
# Dependencies
from flask import Flask, request, jsonify
import joblib
import traceback

# Your API definition
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    if model:
        try:
            json_ = request.json
            print(json_)
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns=model_columns, fill_value=0)

            prediction = list(model.predict(query))

            return jsonify({'prediction': str(prediction)})

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    try:
        port = int(sys.argv[1]) # This is for a command-line input
    except:
        port = 5001 # If you don't provide any port the port will be set to 12345

    lr = joblib.load("recommender_no_rev.pkl") # Load "model.pkl"
    print ('Model loaded')
    model_columns = joblib.load("recommender_columns.pkl") # Load "model_columns.pkl"
    print ('Model columns loaded')

    app.run(port=port, debug=True)

Model loaded
Model columns loaded
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# Load the model that you just saved
lr = joblib.load('recommender_no_rev.pkl')